In [ ]:
import pandas as pd 
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np
from PIL import Image
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
from google.colab import drive

%matplotlib inline

RuntimeError: ignored

In [ ]:
drive.mount('/content/gdrive')  # link google drive


Mounted at /content/gdrive


In [ ]:
label_header = ["photo", "type"]
labels = pd.read_csv("/content/gdrive/My Drive/VRDL_HW1/training_labels.txt",
                     names=label_header, sep=' ')
train_fold = "/content/gdrive/MyDrive/VRDL_HW1/train_images_new"

class_header = ["type"]
class_data = pd.read_csv("/content/gdrive/My Drive/VRDL_HW1/classes.txt",
                         names=class_header, sep=' ')
test_fold = "/content/gdrive/MyDrive/VRDL_HW1/test_images"


NameError: ignored

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_transforms = T.Compose([
    T.Resize((375, 375)),
    T.RandomCrop((350, 350)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize(*imagenet_stats, inplace=True),
    T.RandomErasing(inplace=True)
])

test_transforms = T.Compose([
    T.Resize((350, 350)),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])


In [ ]:
train_dataset = ImageFolder(train_fold, train_transforms)

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b6')
model.fc = torch.nn.Linear(2304, 200)

# use gpu
model = model.cuda()
try:
    model.load_state_dict(torch.load(
        '/content/gdrive/MyDrive/VRDL_HW1/weight_eff.pth'))
    model.eval()
    print('have previous weight')
except:
    print('Cannot load previous weight')



Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b6
have previous weight


In [ ]:
img_order_header = ["name"]
order = pd.read_csv(
        "/content/gdrive/My Drive/VRDL_HW1/testing_img_order.txt",
        names=img_order_header)

from PIL import Image
model.eval()
submission = []
for i in range(len(order)):  # image order is important to your result
    filename = order['name'][i]
    img = Image.open(test_fold + '/' + filename)
    img = test_transforms(img)
    img = torch.unsqueeze(img, 0)
    img = img.cuda()
    pred = model(img)  # the predicted category
    pred_label = torch.argmax(pred, dim=1)
    idx = pred_label[0].item()
    print(idx)
    print(filename, train_dataset.classes[idx])
    tmp = [filename, train_dataset.classes[idx]]
    submission.append(tmp)

np.savetxt('/content/gdrive/My Drive/VRDL_HW1/answer_efficient.txt',
           submission, fmt='%s')
